In [1]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# LCEL
PromptTemplate和ChatPromptTemplate实现了Runnable接口，这意味着它们支持invoke,ainvoke,stream,astream,batch,abatch,astream_log函数

In [3]:
prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}.")
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. You name is {name}"),
        ("human", "{user_input}")
    ]
)

PromptTemplate接受一个字典输入，并返回一个StringPromptValue；
ChatPromptTemplate接受一个字典并返回一个ChatPromptValue.

In [7]:
prompt_val = prompt_template.invoke({"adjective":"funny", "content":"chichens"})
print(type(prompt_val))
print(prompt_val.to_string())
prompt_val.to_messages()

In [9]:
chat_val = chat_template.invoke({"name":"chatgpt", "user_input":"I dont like eating."})
print(chat_val.to_string())
chat_val.to_messages()

## langchain的组合
langchain中的prompts可以相互组合

In [10]:
from langchain.prompts import PromptTemplate

In [13]:
prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"  # 使用加号连接
    + "\n\nand in {language}"
)
prompt

In [15]:
prompt.format(topic="sports", language="spanish")

In [16]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [17]:
prompt = SystemMessage(content="Yout are a nice pirate")
new_prompt = (
    prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
)

In [18]:
new_prompt

In [19]:
new_prompt.format_messages(input="i said hi")

## Few-shot prompt templates

In [20]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

In [22]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Questino: {question}\n{answer}"
)
print(example_prompt.format(**examples[0]))

In [23]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)
print(prompt.format(input="Who was the father of Mary Ball Washington?"))

## Pipeline
Pipeline的作用是将一个Prompt拆分成多个子部分，每个子部分又由一个promt组成。

In [24]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [38]:
full_template = """{introduction}

{example}

{start}
"""
full_prompt = PromptTemplate.from_template(full_template)
introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)
example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

In [39]:
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

In [40]:
pipeline_prompt.input_variables

In [41]:
print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite cat?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)